### EDA 2022
by Harsh Vardhan Pachisia

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
spark = SparkSession.builder.appName('2022-EDA').getOrCreate()

#change configuration settings on Spark 
conf = spark.sparkContext._conf.setAll([('spark.executor.memory', '4g'), ('spark.app.name', 'Spark Updated Conf'), ('spark.executor.cores', '4'), ('spark.cores.max', '4'), ('spark.driver.memory','4g')])

#print spark configuration settings
spark.sparkContext.getConf().getAll()

:: loading settings :: url = jar:file:/usr/lib/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-80b25dc8-e2d3-490f-b4d6-ec450ea01c99;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;4.4.0 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.828 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastutil;7.0.12 in central
	found org.projectlombok#lombok;1.16.8 in central
	found com.google.cloud#google-cloud-storage;2.16.0 in central
	found com.google.guava#guava;31.1-jre in centra

[('spark.app.startTime', '1699642243597'),
 ('spark.stage.maxConsecutiveAttempts', '10'),
 ('spark.dynamicAllocation.minExecutors', '1'),
 ('spark.eventLog.enabled', 'true'),
 ('spark.submit.pyFiles',
  '/root/.ivy2/jars/com.johnsnowlabs.nlp_spark-nlp_2.12-4.4.0.jar,/root/.ivy2/jars/graphframes_graphframes-0.8.2-spark3.1-s_2.12.jar,/root/.ivy2/jars/com.typesafe_config-1.4.2.jar,/root/.ivy2/jars/org.rocksdb_rocksdbjni-6.29.5.jar,/root/.ivy2/jars/com.amazonaws_aws-java-sdk-bundle-1.11.828.jar,/root/.ivy2/jars/com.github.universal-automata_liblevenshtein-3.0.0.jar,/root/.ivy2/jars/com.google.cloud_google-cloud-storage-2.16.0.jar,/root/.ivy2/jars/com.navigamez_greex-1.0.jar,/root/.ivy2/jars/com.johnsnowlabs.nlp_tensorflow-cpu_2.12-0.4.4.jar,/root/.ivy2/jars/it.unimi.dsi_fastutil-7.0.12.jar,/root/.ivy2/jars/org.projectlombok_lombok-1.16.8.jar,/root/.ivy2/jars/com.google.guava_guava-31.1-jre.jar,/root/.ivy2/jars/com.google.guava_failureaccess-1.0.1.jar,/root/.ivy2/jars/com.google.guava_liste

In [3]:
df_2022 = spark.read.csv("gs://msca-bdp-student-gcs/bdp-rideshare-project/rideshare/2022", inferSchema=True, header=True)
df_weather = spark.read.csv("gs://msca-bdp-student-gcs/bdp-rideshare-project/weather/chicago 2020-01-01 to 2022-08-31.csv", inferSchema=True, header=True)
df_2022.printSchema()
df_weather.printSchema()

root
 |-- Trip ID: string (nullable = true)
 |-- Trip Start Timestamp: string (nullable = true)
 |-- Trip End Timestamp: string (nullable = true)
 |-- Trip Seconds: integer (nullable = true)
 |-- Trip Miles: double (nullable = true)
 |-- Pickup Census Tract: long (nullable = true)
 |-- Dropoff Census Tract: long (nullable = true)
 |-- Pickup Community Area: integer (nullable = true)
 |-- Dropoff Community Area: integer (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Tip: integer (nullable = true)
 |-- Additional Charges: double (nullable = true)
 |-- Trip Total: double (nullable = true)
 |-- Shared Trip Authorized: boolean (nullable = true)
 |-- Trips Pooled: integer (nullable = true)
 |-- Pickup Centroid Latitude: double (nullable = true)
 |-- Pickup Centroid Longitude: double (nullable = true)
 |-- Pickup Centroid Location: string (nullable = true)
 |-- Dropoff Centroid Latitude: double (nullable = true)
 |-- Dropoff Centroid Longitude: double (nullable = true)
 |-- Dropof

In [4]:
#display number of records by partition
def displaypartitions(df):
    #number of records by partition
    num = df.rdd.getNumPartitions()
    print("Partitions:", num)
    df.withColumn("partitionId", F.spark_partition_id())\
        .groupBy("partitionId")\
        .count()\
        .orderBy(F.asc("count"))\
        .show(num)

df_2022.rdd.getNumPartitions()


8

In [5]:
displaypartitions(df_2022)

Partitions: 8


+-----------+------+
|partitionId| count|
+-----------+------+
|          7|105933|
|          6|510259|
|          4|510798|
|          5|511378|
|          3|512151|
|          1|513780|
|          2|515744|
|          0|518352|
+-----------+------+



In [7]:
df_2022 = df_2022.repartition(12)
displaypartitions(df_2022)

Partitions: 12


+-----------+------+
|partitionId| count|
+-----------+------+
|         11|308198|
|          2|308198|
|          1|308198|
|          0|308199|
|          9|308200|
|         10|308200|
|          7|308200|
|          3|308200|
|          4|308200|
|          6|308200|
|          8|308201|
|          5|308201|
+-----------+------+



Repartitioned due to unequal number of records in each partition. Now equally balanced. 

In [8]:
df_2022.describe().show()

+-------+----------------+--------------------+--------------------+----------------+-----------------+--------------------+--------------------+---------------------+----------------------+------------------+------------------+------------------+------------------+--------------------+------------------------+-------------------------+------------------------+-------------------------+--------------------------+-------------------------+
|summary|         Trip ID|Trip Start Timestamp|  Trip End Timestamp|    Trip Seconds|       Trip Miles| Pickup Census Tract|Dropoff Census Tract|Pickup Community Area|Dropoff Community Area|              Fare|               Tip|Additional Charges|        Trip Total|        Trips Pooled|Pickup Centroid Latitude|Pickup Centroid Longitude|Pickup Centroid Location|Dropoff Centroid Latitude|Dropoff Centroid Longitude|Dropoff Centroid Location|
+-------+----------------+--------------------+--------------------+----------------+-----------------+-----------

**Missing Values**

In [10]:
#Find the number of missing values for each column
from pyspark.sql.functions import isnan, when, count, col
# don't run for loops on the data, running for loops on the columns is fine, doing it on the entire data can cause problems. 
df_2022.select([count(when(df_2022[c].isNull(), c)).alias(c) for c in df_2022.columns]).show()

+-------+--------------------+------------------+------------+----------+-------------------+--------------------+---------------------+----------------------+----+---+------------------+----------+----------------------+------------+------------------------+-------------------------+------------------------+-------------------------+--------------------------+-------------------------+
|Trip ID|Trip Start Timestamp|Trip End Timestamp|Trip Seconds|Trip Miles|Pickup Census Tract|Dropoff Census Tract|Pickup Community Area|Dropoff Community Area|Fare|Tip|Additional Charges|Trip Total|Shared Trip Authorized|Trips Pooled|Pickup Centroid Latitude|Pickup Centroid Longitude|Pickup Centroid Location|Dropoff Centroid Latitude|Dropoff Centroid Longitude|Dropoff Centroid Location|
+-------+--------------------+------------------+------------+----------+-------------------+--------------------+---------------------+----------------------+----+---+------------------+----------+----------------------

In [11]:
#number of trips in 2022
df_2022.distinct().count()

3698395

In [12]:
# number of trips in 2022 without any na's
df_2022.dropna(how='any').count()

1822152

In [13]:
1822152/3698395

0.49268723324577285

Have only around 50% of trips with no na's. 

In [14]:
df_2022 = df_2022.dropna(how='any')
df_2022 = df_2022.dropDuplicates()

In [15]:
df_2022.printSchema()

root
 |-- Trip ID: string (nullable = true)
 |-- Trip Start Timestamp: string (nullable = true)
 |-- Trip End Timestamp: string (nullable = true)
 |-- Trip Seconds: integer (nullable = true)
 |-- Trip Miles: double (nullable = true)
 |-- Pickup Census Tract: long (nullable = true)
 |-- Dropoff Census Tract: long (nullable = true)
 |-- Pickup Community Area: integer (nullable = true)
 |-- Dropoff Community Area: integer (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Tip: integer (nullable = true)
 |-- Additional Charges: double (nullable = true)
 |-- Trip Total: double (nullable = true)
 |-- Shared Trip Authorized: boolean (nullable = true)
 |-- Trips Pooled: integer (nullable = true)
 |-- Pickup Centroid Latitude: double (nullable = true)
 |-- Pickup Centroid Longitude: double (nullable = true)
 |-- Pickup Centroid Location: string (nullable = true)
 |-- Dropoff Centroid Latitude: double (nullable = true)
 |-- Dropoff Centroid Longitude: double (nullable = true)
 |-- Dropof

In [17]:
df_2022.show(2, truncate = False)

+----------------------------------------+----------------------+----------------------+------------+----------+-------------------+--------------------+---------------------+----------------------+----+---+------------------+----------+----------------------+------------+------------------------+-------------------------+------------------------------------+-------------------------+--------------------------+------------------------------------+
|Trip ID                                 |Trip Start Timestamp  |Trip End Timestamp    |Trip Seconds|Trip Miles|Pickup Census Tract|Dropoff Census Tract|Pickup Community Area|Dropoff Community Area|Fare|Tip|Additional Charges|Trip Total|Shared Trip Authorized|Trips Pooled|Pickup Centroid Latitude|Pickup Centroid Longitude|Pickup Centroid Location            |Dropoff Centroid Latitude|Dropoff Centroid Longitude|Dropoff Centroid Location           |
+----------------------------------------+----------------------+----------------------+--------

In [18]:
# Convert 'Trip start timestamp' and 'Trip End Timestamp' columns to timestamp
df_2022 = df_2022.withColumn("Trip Start Timestamp", F.to_timestamp(F.col("Trip Start Timestamp"), "MM/dd/yyyy hh:mm:ss a"))
df_2022 = df_2022.withColumn("Trip End Timestamp", F.to_timestamp(F.col("Trip End Timestamp"), "MM/dd/yyyy hh:mm:ss a"))

In [19]:
df_2022.printSchema()

root
 |-- Trip ID: string (nullable = true)
 |-- Trip Start Timestamp: timestamp (nullable = true)
 |-- Trip End Timestamp: timestamp (nullable = true)
 |-- Trip Seconds: integer (nullable = true)
 |-- Trip Miles: double (nullable = true)
 |-- Pickup Census Tract: long (nullable = true)
 |-- Dropoff Census Tract: long (nullable = true)
 |-- Pickup Community Area: integer (nullable = true)
 |-- Dropoff Community Area: integer (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Tip: integer (nullable = true)
 |-- Additional Charges: double (nullable = true)
 |-- Trip Total: double (nullable = true)
 |-- Shared Trip Authorized: boolean (nullable = true)
 |-- Trips Pooled: integer (nullable = true)
 |-- Pickup Centroid Latitude: double (nullable = true)
 |-- Pickup Centroid Longitude: double (nullable = true)
 |-- Pickup Centroid Location: string (nullable = true)
 |-- Dropoff Centroid Latitude: double (nullable = true)
 |-- Dropoff Centroid Longitude: double (nullable = true)
 |-- 

In [29]:
df_2022 = df_2022.drop('Trips Pooled','Additional Charges','Shared Trip Authorized', "Fare", "Tip")
df_2022 = df_2022.withColumnRenamed("Trip ID","ID").withColumnRenamed("Trip Start Timestamp","start_timestamp").withColumnRenamed("Trip End Timestamp","end_timestamp").withColumnRenamed("Trip Miles",\
                "miles").withColumnRenamed("Pickup Census Tract","pickup_tract").withColumnRenamed("Dropoff Census Tract","dropoff_tract").withColumnRenamed("Pickup Community Area","pickup_area"\
                ).withColumnRenamed("Dropoff Community Area","dropoff_area").withColumnRenamed("Trip Total","total").withColumnRenamed("Pickup Centroid Latitude","pickup_lat").withColumnRenamed(\
                "Pickup Centroid Longitude","pickup_lon").withColumnRenamed("Pickup Centroid Location","pickup_location").withColumnRenamed("Dropoff Centroid Latitude","dropoff_lat").withColumnRenamed(\
                "Dropoff Centroid Longitude","dropoff_lon").withColumnRenamed("Dropoff Centroid Latitude", "dropoff_lat").withColumnRenamed("Trip Seconds", "trip_seconds").withColumnRenamed("Dropoff Centroid Location", "dropoff_location")

In [30]:
df_2022.printSchema()

root
 |-- ID: string (nullable = true)
 |-- start_timestamp: timestamp (nullable = true)
 |-- end_timestamp: timestamp (nullable = true)
 |-- trip_seconds: integer (nullable = true)
 |-- miles: double (nullable = true)
 |-- pickup_tract: long (nullable = true)
 |-- dropoff_tract: long (nullable = true)
 |-- pickup_area: integer (nullable = true)
 |-- dropoff_area: integer (nullable = true)
 |-- total: double (nullable = true)
 |-- pickup_lat: double (nullable = true)
 |-- pickup_lon: double (nullable = true)
 |-- pickup_location: string (nullable = true)
 |-- dropoff_lat: double (nullable = true)
 |-- dropoff_lon: double (nullable = true)
 |-- dropoff_location: string (nullable = true)



In [32]:
df_weather = df_weather.withColumn('datetime',F.to_date(df_weather['datetime'], "yyyy-mm-dd"))

In [33]:
hp_census_tracts_2010_2020 = [17031411000,17031410900,17031410100,17031411100,17031410800,17031410200,17031410700,17031411200,17031836200,17031410600,17031836300,17031410500,
                    17031410300,17031410400,17031410600,17031410800,17031411300,17031411400]
df_hp = df_2022.filter((df_2022.pickup_tract.isin(hp_census_tracts_2010_2020)) & (df_2022.dropoff_tract.isin(hp_census_tracts_2010_2020)))

In [43]:
df_2022.select(['pickup_tract']).distinct().show()

+------------+
|pickup_tract|
+------------+
| 17031832600|
| 17031837400|
| 17031831300|
| 17031560300|
| 17031530503|
| 17031650500|
| 17031710700|
| 17031271300|
| 17031062200|
| 17031843200|
| 17031430500|
| 17031020602|
| 17031130200|
| 17031230200|
| 17031241300|
| 17031838000|
| 17031150501|
| 17031351500|
| 17031440800|
| 17031291200|
+------------+
only showing top 20 rows



In [48]:
df_2022 = df_2022.withColumn('date_only', F.to_date(df_2022.end_timestamp))
#df_2022.select('end_timestamp').distinct().show(10)

In [50]:
df_2022 = df_2022.withColumn('date_only', F.to_date(df_2022.end_timestamp))
df_2022.select('date_only').distinct().show(100)

+----------+
| date_only|
+----------+
|2022-01-09|
|2022-01-15|
|2022-01-18|
|2022-01-10|
|2022-01-08|
|2022-01-24|
|2022-01-23|
|2022-01-16|
|2022-01-13|
|2022-01-06|
|2022-01-03|
|2022-01-25|
|2022-01-07|
|2022-01-22|
|2022-01-04|
|2022-01-17|
|2022-01-20|
|2022-01-05|
|2022-01-14|
|2022-01-19|
|2022-01-12|
|2022-01-01|
|2022-01-21|
|2022-01-11|
|2022-01-02|
+----------+

